In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
from torch.utils.data import Dataset
import os
import tifffile as tiff
import numpy as np
from torchvision import transforms
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F

from collections import Counter

from scipy.stats import mode



In [3]:
class ImageDataset(Dataset):
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.image_files = []
        self.labels = []
        self.class_names = []  # To store class (folder) names
        self.class_to_idx = {}  # To map class names to indices
        self.class_counts = {}  # To store count of images per class

        # Define a transformation to resize the images
        self.transform = transforms.Compose([transforms.Resize((96, 96))])

        # Get all subdirectories (classes)
        self.class_names = [d for d in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, d))]
        self.class_names.sort()  # Ensure consistent ordering
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.class_names)}

        # Load images and assign labels based on folder name
        for class_name in self.class_names:
            class_dir = os.path.join(image_dir, class_name)
            class_images = [os.path.join(class_dir, file) for file in os.listdir(class_dir) if file.endswith(('.tiff', '.tif'))]
            self.image_files.extend(class_images)
            self.labels.extend([self.class_to_idx[class_name]] * len(class_images))  # Assign class index as label
            
            # Store the count of images per class
            self.class_counts[class_name] = len(class_images)

        # Print labels and counts
        self.print_labels_and_counts()
     
    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        label = self.labels[idx]

        image = tiff.imread(img_path)

        # Ensure the image has 3 layers (channels)
        if image.shape[0] != 3:
            raise ValueError(f"Image {img_path} does not have exactly 3 layers.")
        
        # Normalize the 16-bit image to [0, 1]
        original_image = image.astype(np.float32) / 65535.0

        # Convert to a torch tensor 
        image = torch.tensor(original_image, dtype=torch.float32)
        #print(image.shape)
        # Apply the resizing transform (resize)
        image = self.transform(image)

        flattened_image = image.view(-1).numpy()  # Reshape to (96*96*3, )
        
        return flattened_image, label

    def print_labels_and_counts(self):
        print("Class Labels and Counts:")
        for class_name, class_index in self.class_to_idx.items():
            print(f"Class '{class_name}' (Index {class_index}): {self.class_counts[class_name]} images")




image_dir = r"G:\cluster_40\cure"
#image_dir = r"G:\cluster_40\uncure"
#image_dir=  r"G:\classification\cond_all"
#image_dir=  r"G:\classification\cure_cond" 


dataset = ImageDataset(image_dir)  # Initialize dataset, it will automatically print labels and counts


Class Labels and Counts:
Class 'cure_cond7_40' (Index 0): 40 images
Class 'ex' (Index 1): 40 images
Class 'sdonly_40' (Index 2): 40 images


In [4]:
for i in range(len(dataset)):
    falttened_image,l = dataset[i]
    print(falttened_image.shape)
    print(l)  
    break

(27648,)
0


In [5]:
type(falttened_image)

numpy.ndarray

In [6]:
falttened_image.shape

(27648,)

In [7]:
# Combine all images into a single dataset for K-Means input
def load_image_data(dataset):
    all_images = []
    all_labels = []
    
    for i in range(len(dataset)):
        flattened_image, label = dataset[i]  # Unpack the image and label
        all_images.append(flattened_image)
        all_labels.append(label)
    
    # Stack all the flattened images to create a large dataset
    # Convert to numpy arrays for K-Means input
    return np.vstack(all_images), np.array(all_labels)

In [8]:
X, labels = load_image_data(dataset) 

In [9]:
X.shape

(120, 27648)

In [10]:
print(type(labels))

labels.shape

<class 'numpy.ndarray'>


(120,)

In [11]:

# K-Means initialization function
def kMeans_init_centroids(X, K):
    randidx = np.random.permutation(X.shape[0])
    centroids = X[randidx[:K]]
    return centroids

# Function to find the closest centroids
def find_closest_centroids(X, centroids):
    idx = np.zeros(X.shape[0], dtype=int)
    for i in range(X.shape[0]):
        distances = np.linalg.norm(X[i] - centroids, axis=1)
        idx[i] = np.argmin(distances)
    return idx

# Function to compute new centroids
def compute_centroids(X, idx, K):
    centroids = np.zeros((K, X.shape[1]))  # Use shape[1] for features
    for k in range(K):
        points = X[idx == k]
        centroids[k] = np.mean(points, axis=0) if len(points) > 0 else centroids[k]
    return centroids

def run_kMeans(X, initial_centroids, max_iters=10):
    K = initial_centroids.shape[0]
    centroids = initial_centroids
    idx = np.zeros(X.shape[0])

    for i in range(max_iters):
        
        # Assign each data point to the closest centroid
        idx = find_closest_centroids(X, centroids)
        
        # Compute new centroids
        centroids = compute_centroids(X, idx, K)
    return centroids,idx

def calculate_accuracy(idx, labels_np, K=3):
    """Calculate accuracy for a given set of cluster assignments."""
    total_samples = len(labels_np)
    cluster_label_counts = {}
    label_to_cluster = {}
    correctly_classified = 0

    # Count true labels in each cluster
    for cluster in range(K):
        cluster_indices = np.where(idx == cluster)[0]
        cluster_labels = labels_np[cluster_indices]
        label_counts = Counter(cluster_labels)
        cluster_label_counts[cluster] = label_counts
    
    # Assign each label to the cluster where it is most common
    for label in range(3):  # Assuming 3 classes: 0, 1, 2
        max_count = 0
        assigned_cluster = None
        for cluster, label_counts in cluster_label_counts.items():
            if label_counts[label] > max_count:
                max_count = label_counts[label]
                assigned_cluster = cluster
        if assigned_cluster is not None:
            label_to_cluster[label] = assigned_cluster
            correctly_classified += max_count

    accuracy = correctly_classified / total_samples
    return accuracy, cluster_label_counts 

def main_kMeans_experiment(X, labels_np, K=3, num_runs=100, max_iters=50):
    accuracies = []
    best_accuracy = 0
    best_cluster_counts = None
    best_centroids = None
    best_idx = None

    for run in range(num_runs):
        # Initialize centroids randomly
        initial_centroids = kMeans_init_centroids(X, K)
        
        # Run K-Means
        centroids, idx = run_kMeans(X, initial_centroids, max_iters)
        
        # Calculate accuracy
        accuracy, cluster_counts = calculate_accuracy(idx, labels_np, K)
        accuracies.append(accuracy)
        
        # Track the best run
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_cluster_counts = cluster_counts
            best_centroids = centroids
            best_idx = idx
        
        print(f"Run {run + 1}/{num_runs}, Accuracy: {accuracy:.4f}")

    # Print final results
    print("\n=== Final Results ===")
    print(f"Max Accuracy: {best_accuracy:.4f}")
    print("Cluster Descriptions:")
    for cluster, counts in best_cluster_counts.items():
        print(f"Cluster {cluster} has true labels: {counts}")
    
    # Return the best centroids and idx along with accuracy
    return best_accuracy, best_centroids, best_idx, best_cluster_counts


K = 3  # Number of clusters
num_runs = 400  # Number of K-Means runs
max_iters = 50  # Maximum iterations per run

best_accuracy, best_centroids, best_idx, best_cluster_counts = main_kMeans_experiment(X, labels, K, num_runs, max_iters)

Run 1/400, Accuracy: 0.5833
Run 2/400, Accuracy: 0.7083
Run 3/400, Accuracy: 0.7583
Run 4/400, Accuracy: 0.6167
Run 5/400, Accuracy: 0.7167
Run 6/400, Accuracy: 0.7333
Run 7/400, Accuracy: 0.7167
Run 8/400, Accuracy: 0.7333
Run 9/400, Accuracy: 0.7167
Run 10/400, Accuracy: 0.7083
Run 11/400, Accuracy: 0.7167
Run 12/400, Accuracy: 0.7083
Run 13/400, Accuracy: 0.5833
Run 14/400, Accuracy: 0.5833
Run 15/400, Accuracy: 0.7750
Run 16/400, Accuracy: 0.7167
Run 17/400, Accuracy: 0.7333
Run 18/400, Accuracy: 0.6167
Run 19/400, Accuracy: 0.5417
Run 20/400, Accuracy: 0.5417
Run 21/400, Accuracy: 0.5833
Run 22/400, Accuracy: 0.5833
Run 23/400, Accuracy: 0.7167
Run 24/400, Accuracy: 0.7750
Run 25/400, Accuracy: 0.5833
Run 26/400, Accuracy: 0.6000
Run 27/400, Accuracy: 0.7750
Run 28/400, Accuracy: 0.5833
Run 29/400, Accuracy: 0.7250
Run 30/400, Accuracy: 0.6000
Run 31/400, Accuracy: 0.6083
Run 32/400, Accuracy: 0.7167
Run 33/400, Accuracy: 0.6000
Run 34/400, Accuracy: 0.6083
Run 35/400, Accuracy: 0

KeyboardInterrupt: 

In [11]:
(33+22+37)/120

0.7666666666666667

In [13]:
(101+126)/423

0.5366430260047281

In [12]:

# K-Means initialization function
def kMeans_init_centroids(X, K):
    randidx = np.random.permutation(X.shape[0])
    centroids = X[randidx[:K]]
    return centroids

# Function to find the closest centroids
def find_closest_centroids(X, centroids):
    idx = np.zeros(X.shape[0], dtype=int)
    for i in range(X.shape[0]):
        distances = np.linalg.norm(X[i] - centroids, axis=1)
        idx[i] = np.argmin(distances)
    return idx

# Function to compute new centroids
def compute_centroids(X, idx, K):
    centroids = np.zeros((K, X.shape[1]))  # Use shape[1] for features
    for k in range(K):
        points = X[idx == k]
        centroids[k] = np.mean(points, axis=0) if len(points) > 0 else centroids[k]
    return centroids

def run_kMeans(X, initial_centroids, max_iters=10):
    K = initial_centroids.shape[0]
    centroids = initial_centroids
    idx = np.zeros(X.shape[0])

    for i in range(max_iters):
        
        # Assign each data point to the closest centroid
        idx = find_closest_centroids(X, centroids)
        
        # Compute new centroids
        centroids = compute_centroids(X, idx, K)
    return centroids,idx

def calculate_accuracy(idx, labels_np, K=3):
    """Calculate accuracy for a given set of cluster assignments."""
    total_samples = len(labels_np)
    cluster_label_counts = {}
    label_to_cluster = {}
    correctly_classified = 0

    # Count true labels in each cluster
    for cluster in range(K):
        cluster_indices = np.where(idx == cluster)[0]
        cluster_labels = labels_np[cluster_indices]
        label_counts = Counter(cluster_labels)
        cluster_label_counts[cluster] = label_counts
    
    # Track clusters already assigned to labels
    assigned_clusters = set()

    # Assign each label to the cluster where it is most common, respecting the rule
    for label in range(3):  # Assuming 3 classes: 0, 1, 2
        max_count = 0
        assigned_cluster = None
        for cluster, label_counts in cluster_label_counts.items():
            # Skip clusters already assigned to another label
            if cluster in assigned_clusters:
                continue
            if label_counts[label] > max_count:
                max_count = label_counts[label]
                assigned_cluster = cluster
        # Assign this label to the cluster and mark the cluster as used
        if assigned_cluster is not None:
            label_to_cluster[label] = assigned_cluster
            assigned_clusters.add(assigned_cluster)
            correctly_classified += max_count

    accuracy = correctly_classified / total_samples
    return accuracy, cluster_label_counts 

def main_kMeans_experiment(X, labels_np, K=3, num_runs=100, max_iters=50):
    accuracies = []
    best_accuracy = 0
    best_cluster_counts = None
    best_centroids = None
    best_idx = None

    for run in range(num_runs):
        # Initialize centroids randomly
        initial_centroids = kMeans_init_centroids(X, K)
        
        # Run K-Means
        centroids, idx = run_kMeans(X, initial_centroids, max_iters)
        
        # Calculate accuracy
        accuracy, cluster_counts = calculate_accuracy(idx, labels_np, K)
        accuracies.append(accuracy)
        
        # Track the best run
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_cluster_counts = cluster_counts
            best_centroids = centroids
            best_idx = idx
        
        print(f"Run {run + 1}/{num_runs}, Accuracy: {accuracy:.4f}")

    # Print final results
    print("\n=== Final Results ===")
    print(f"Max Accuracy: {best_accuracy:.4f}")
    print("Cluster Descriptions:")
    for cluster, counts in best_cluster_counts.items():
        print(f"Cluster {cluster} has true labels: {counts}")
    
    # Return the best centroids and idx along with accuracy
    return best_accuracy, best_centroids, best_idx, best_cluster_counts


K = 3  # Number of clusters
num_runs = 400  # Number of K-Means runs
max_iters = 50  # Maximum iterations per run

best_accuracy, best_centroids, best_idx, best_cluster_counts = main_kMeans_experiment(X, labels, K, num_runs, max_iters)

Run 1/400, Accuracy: 0.3688
Run 2/400, Accuracy: 0.3688
Run 3/400, Accuracy: 0.3688
Run 4/400, Accuracy: 0.3688
Run 5/400, Accuracy: 0.3688
Run 6/400, Accuracy: 0.3688
Run 7/400, Accuracy: 0.3688
Run 8/400, Accuracy: 0.3688
Run 9/400, Accuracy: 0.3688
Run 10/400, Accuracy: 0.3688
Run 11/400, Accuracy: 0.3688
Run 12/400, Accuracy: 0.3688
Run 13/400, Accuracy: 0.3688
Run 14/400, Accuracy: 0.3688
Run 15/400, Accuracy: 0.3688
Run 16/400, Accuracy: 0.3688
Run 17/400, Accuracy: 0.3688
Run 18/400, Accuracy: 0.3688
Run 19/400, Accuracy: 0.3688
Run 20/400, Accuracy: 0.3688
Run 21/400, Accuracy: 0.3688
Run 22/400, Accuracy: 0.3688
Run 23/400, Accuracy: 0.3688
Run 24/400, Accuracy: 0.3688
Run 25/400, Accuracy: 0.3688
Run 26/400, Accuracy: 0.3688
Run 27/400, Accuracy: 0.3688
Run 28/400, Accuracy: 0.3688
Run 29/400, Accuracy: 0.3688
Run 30/400, Accuracy: 0.3688
Run 31/400, Accuracy: 0.3688
Run 32/400, Accuracy: 0.3688
Run 33/400, Accuracy: 0.3688
Run 34/400, Accuracy: 0.3688
Run 35/400, Accuracy: 0

KeyboardInterrupt: 

In [ ]:
(33+22+36)/120

In [ ]:
(100+230+10)/615

In [ ]:
s

from scipy.stats import mode
import numpy as np

# Combine all images into a single dataset for K-Means input
def load_image_data(dataset):
    all_images = []
    all_labels = []
    
    for i in range(len(dataset)):
        flattened_image, label = dataset[i]  # Unpack the image and label
        all_images.append(flattened_image)
        all_labels.append(label)
    
    # Stack all the flattened images to create a large dataset
    # Convert to numpy arrays for K-Means input
    return np.vstack(all_images), np.array(all_labels)

# K-Means initialization function
def kMeans_init_centroids(X, K):
    randidx = np.random.permutation(X.shape[0])
    centroids = X[randidx[:K]]
    return centroids

# Function to find the closest centroids
def find_closest_centroids(X, centroids):
    idx = np.zeros(X.shape[0], dtype=int)
    for i in range(X.shape[0]):
        distances = np.linalg.norm(X[i] - centroids, axis=1)
        idx[i] = np.argmin(distances)
    return idx

# Function to compute new centroids
def compute_centroids(X, idx, K):
    centroids = np.zeros((K, X.shape[1]))  # Use shape[1] for features
    for k in range(K):
        points = X[idx == k]
        centroids[k] = np.mean(points, axis=0) if len(points) > 0 else centroids[k]
    return centroids

# Function to run K-Means algorithm with cost tracking
def run_kMeans(X, initial_centroids, max_iters=10):
    K = initial_centroids.shape[0]
    centroids = initial_centroids
    idx = np.zeros(X.shape[0])

    for i in range(max_iters):
        #print(f"K-Means iteration {i}/{max_iters - 1}")
        
        # Assign each data point to the closest centroid
        idx = find_closest_centroids(X, centroids)
        
        # Compute new centroids
        centroids = compute_centroids(X, idx, K)
        
    return centroids, idx

# Function to reorder K-Means labels to match true labels
def reorder_labels(true_labels, predicted_labels):
    reordered_labels = np.zeros_like(predicted_labels)

    for cluster in np.unique(predicted_labels):
        mask = (predicted_labels == cluster)
        # Use mode and handle cases where mode() returns a scalar
        most_common_label = mode(true_labels[mask], axis=None).mode  # Get the mode for the current cluster
        if isinstance(most_common_label, np.ndarray):
            most_common_label = most_common_label[0]  # Safely extract the mode value if it's an array
        
        reordered_labels[mask] = most_common_label

    return reordered_labels

# Run the K-Means algorithm 100 times with different random initializations
max_accuracy = 0
num_runs = 40

for run in range(num_runs):
    print(f"Run {run + 1}/{num_runs}")
    
    # Load dataset (replace `dataset` with your actual dataset)
    X, labels = load_image_data(dataset)
    
    K = 3  # Number of clusters
    initial_centroids = kMeans_init_centroids(X, K)  # Initialize centroids
    max_iters = 50  # Maximum iterations
    
    # Run K-Means
    centroids, idx = run_kMeans(X, initial_centroids, max_iters)
    
    # Reorder labels and calculate accuracy
    reordered_idx = reorder_labels(labels, idx)
    accuracy = np.sum(reordered_idx == labels) / len(labels) * 100
    
    print(f"Accuracy for run {run + 1}: {accuracy:.2f}%")
    max_accuracy = max(max_accuracy, accuracy)

print(f"Maximum accuracy achieved over {num_runs} runs: {max_accuracy:.2f}%")


In [136]:
# Combine all images into a single dataset for K-Means input
def load_image_data(dataset):
    all_images = []
    all_labels = []
    
    for i in range(len(dataset)):
        flattened_image, label = dataset[i]  # Unpack the image and label
        all_images.append(flattened_image)
        all_labels.append(label)
    
    # Stack all the flattened images to create a large dataset
    # Convert to numpy arrays for K-Means input
    return np.vstack(all_images), np.array(all_labels)

# K-Means initialization function
def kMeans_init_centroids(X, K):
    randidx = np.random.permutation(X.shape[0])
    centroids = X[randidx[:K]]
    return centroids

# Function to find the closest centroids
def find_closest_centroids(X, centroids):
    idx = np.zeros(X.shape[0], dtype=int)
    for i in range(X.shape[0]):
        distances = np.linalg.norm(X[i] - centroids, axis=1)
        idx[i] = np.argmin(distances)
    return idx

# Function to compute new centroids
def compute_centroids(X, idx, K):
    centroids = np.zeros((K, X.shape[1]))  # Use shape[1] for features
    for k in range(K):
        points = X[idx == k]
        centroids[k] = np.mean(points, axis=0) if len(points) > 0 else centroids[k]
    return centroids

# Function to run K-Means algorithm with cost tracking
def run_kMeans(X, initial_centroids, max_iters=10):
    K = initial_centroids.shape[0]
    centroids = initial_centroids
    idx = np.zeros(X.shape[0])
    
    #best_centroids = centroids
    #lowest_cost = float('inf')
    #final_iteration = 0  # To keep track of the iteration where the best centroids were found
    #best_idx = idx

    for i in range(max_iters):
        #print(f"K-Means iteration {i}/{max_iters - 1}")
        
        # Assign each data point to the closest centroid
        idx = find_closest_centroids(X, centroids)
        
        # Compute new centroids
        centroids = compute_centroids(X, idx, K)

        # Calculate cost function for the current centroids
        cost = np.sum(np.linalg.norm(X - centroids[idx], axis=1)**2)
        #print(f"Cost function value: {cost:.4f}")  # Print the cost function value

        # Check if this is the best cost so far
        #if cost < lowest_cost:
            #lowest_cost = cost
            #best_centroids = centroids
            #best_idx = idx.copy()
            #final_iteration = i  # Update the iteration where best centroids were found

    #print(f"Final centroids selected from iteration: {final_iteration}")  # Indicate which iteration was chosen
    return centroids,idx
    #return best_centroids, best_idx, centroids,idx                                    # Return the best centroids



In [ ]:
# Main function to run the K-Means algorithm
X, labels = load_image_data(dataset)  # Load data and labels 

# To check and print the dimensions:
#print("Shape of stacked images (X):", X.shape)
#print("Shape of labels:", labels.shape)  

K = 3                          # Step 2: Set number of clusters
initial_centroids = kMeans_init_centroids(X, K)  # Step 3: Initialize centroids 
max_iters = 50                # Step 4: Number of iterations

centroids, idx = run_kMeans(X, initial_centroids, max_iters)
#best_centroids, best_idx, centroids, idx = run_kMeans(X, initial_centroids, max_iters)  # Step 5: Run K-Means
#print("Final centroids:", best_centroids)  # Output the final centroids
# `idx` contains the cluster assignments from KMeans
#  `labels` contains the true labels

K = len(np.unique(idx))  # Number of clusters

# Iterate over each cluster and count the true labels in that cluster
for cluster in range(K):
    # Find the indices of images assigned to the current cluster
    cluster_indices = np.where(idx == cluster)[0]
    
    # Get the true labels for the images in this cluster
    cluster_labels = labels[cluster_indices]
    
    # Use Counter to count occurrences of each label in the cluster
    label_counts = Counter(cluster_labels)
    
    # Print the result
    print(f"Cluster {cluster} has true labels: {label_counts}")


In [ ]:
num = 126+101
print(num)

In [ ]:
num/423

In [140]:
# Function to reorder K-Means labels to match true labels
def reorder_labels(true_labels, predicted_labels):
    reordered_labels = np.zeros_like(predicted_labels)

    for cluster in np.unique(predicted_labels):
        mask = (predicted_labels == cluster)
        # Use mode and handle cases where mode() returns a scalar
        most_common_label = mode(true_labels[mask], axis=None).mode  # Get the mode for the current cluster
        if isinstance(most_common_label, np.ndarray):
            most_common_label = most_common_label[0]  # Safely extract the mode value if it's an array
        
        reordered_labels[mask] = most_common_label

    return reordered_labels


In [ ]:
# Apply this after running K-Means
reordered_idx = reorder_labels(labels, idx)

# Now you can compare `reordered_idx` with `labels` to evaluate accuracy
accuracy = np.sum(reordered_idx == labels) / len(labels) * 100
print(f"Accuracy: {accuracy:.2f}%")

print("Cluster assignments:", idx)
print("True labels:", labels)

print("Cluster assignments:", best_idx)
print("True labels:", labels)

In [ ]:
best_idx == idx

K = len(np.unique(best_idx))  # Number of clusters

# Iterate over each cluster and count the true labels in that cluster
for cluster in range(K):
    # Find the indices of images assigned to the current cluster
    cluster_indices = np.where(best_idx == cluster)[0]
    
    # Get the true labels for the images in this cluster
    cluster_labels = labels[cluster_indices]
    
    # Use Counter to count occurrences of each label in the cluster
    label_counts = Counter(cluster_labels)
    
    # Print the result
    print(f"Cluster {cluster} has true labels: {label_counts}")

# Apply this after running K-Means
reordered_idx = reorder_labels(labels, best_idx)

# Now you can compare `reordered_idx` with `labels` to evaluate accuracy
accuracy = np.sum(reordered_idx == labels) / len(labels) * 100
print(f"Accuracy: {accuracy:.2f}%")

In [143]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [144]:
# Function to reduce the dimensionality of data points and centroids to 2D using PCA
def reduce_to_2D_pca(X, centroids):
    # Initialize PCA with 2 components
    pca = PCA(n_components=2)
    
    # Apply PCA to the data points (X) and centroids
    X_2D = pca.fit_transform(X)  # Reducing original data points
    centroids_2D = pca.transform(centroids)  # Reducing centroids

    return X_2D, centroids_2D

# Function to plot 2D visualization of clustered data points and centroids with true labels
def plot_2D_clusters_with_labels(X_2D, centroids_2D, idx, labels, K):
    plt.figure(figsize=(10, 7))
    
    # Define color map for clusters
    colors = plt.cm.rainbow(np.linspace(0, 1, K))
    
    # Plot data points colored by their cluster assignment
    for k in range(K):
        cluster_points = X_2D[idx == k]
        cluster_labels = labels[idx == k]  # Get true labels for the current cluster
        
        # Scatter plot for each cluster
        plt.scatter(cluster_points[:, 0], cluster_points[:, 1], 
                    c=[colors[k]], label=f"Cluster {k+1}", alpha=0.6)
        
        # Annotate each point with its true label
        for i in range(cluster_points.shape[0]):
            plt.annotate(str(cluster_labels[i]), 
                         (cluster_points[i, 0], cluster_points[i, 1]), 
                         fontsize=8, alpha=0.75)

    # Plot centroids as larger markers
    plt.scatter(centroids_2D[:, 0], centroids_2D[:, 1], 
                c='k', marker='x', s=200, label='Centroids')

    plt.title("2D Visualization of Clusters using PCA with True Labels")
    plt.xlabel("Component 1")
    plt.ylabel("Component 2")
    plt.legend()
    plt.grid()
    plt.show()


In [ ]:
# Apply PCA after K-Means clustering for 2D visualization
X_2D_pca, centroids_2D_pca = reduce_to_2D_pca(X, centroids)

# Plot the 2D clusters with centroids and true labels
plot_2D_clusters_with_labels(X_2D_pca, centroids_2D_pca, idx, labels, K)


# Apply PCA after K-Means clustering for 2D visualization
X_2D_pca, centroids_2D_pca = reduce_to_2D_pca(X, best_centroids)

# Plot the 2D clusters with centroids and true labels
plot_2D_clusters_with_labels(X_2D_pca, centroids_2D_pca, best_idx, labels, K)


from sklearn.preprocessing import normalize

# Combine all images into a single dataset for K-Means input
def load_image_data(dataset):
    all_images = []
    all_labels = []
    
    for i in range(len(dataset)):
        flattened_image, label = dataset[i]  # Unpack the image and label
        all_images.append(flattened_image)
        all_labels.append(label)
    
    # Stack all the flattened images to create a large dataset
    # Convert to numpy arrays for K-Means input
    return np.vstack(all_images), np.array(all_labels)

# K-Means initialization function
def kMeans_init_centroids(X, K):
    randidx = np.random.permutation(X.shape[0])
    centroids = X[randidx[:K]]
    return centroids

# Function to find the closest centroids using Cosine Similarity
def find_closest_centroids(X, centroids):
    # Normalize both the data points and centroids to ensure we compute cosine similarity
    X_norm = normalize(X, axis=1)
    centroids_norm = normalize(centroids, axis=1)
    #print(X_norm.shape)              #(60, 27648)
    #print(centroids_norm.shape)  #(2, 27648)
    #print(X_norm[0].shape) #(27648,)
    
    # Assign data points to closest centroids based on cosine similarity
    idx = np.zeros(X.shape[0], dtype=int)
    for i in range(X.shape[0]):
        # Compute cosine similarity
        similarities = np.dot(centroids_norm, X_norm[i])  # Dot product gives cosine similarity
        #print(similarities.shape)
        idx[i] = np.argmax(similarities)  # We want the most similar (highest value)
    return idx  #idx = [1, 0, 1, 1, 0,...]


def compute_centroids(X, idx, K):
    centroids = np.zeros((K, X.shape[1]))  # Use shape[1] for features
    for k in range(K):
        points = X[idx == k]
        if len(points) > 0:
            # Normalize the points within the cluster along each feature axis
            normalized_points = normalize(points, axis=1)
            # Compute the mean of the normalized points to get the new centroid
            centroids[k] = np.mean(normalized_points, axis=0)
    return centroids

# Function to run K-Means algorithm with cost tracking (using Cosine Similarity)
def run_kMeans(X, initial_centroids, max_iters=10):
    K = initial_centroids.shape[0]
    centroids = initial_centroids
    idx = np.zeros(X.shape[0])
    
    best_centroids = centroids
    lowest_cost = float('inf')
    final_iteration = 0  # To keep track of the iteration where the best centroids were found
    best_idx = idx  # Track the best index assignment

    for i in range(max_iters):
        print(f"K-Means iteration {i}/{max_iters - 1}")
        
        # Assign each data point to the closest centroid using cosine similarity
        idx = find_closest_centroids(X, centroids)
        
        
        # Compute new centroids
        centroids = compute_centroids(X, idx, K)

        #Range of cosine distance is from 0 to 2, 0 — identical vectors, 1 — no correlation, 2 — absolutely different.

        # Calculate cost function for the current centroids using cosine distance
        X_norm = normalize(X, axis=1)
        centroids_norm = normalize(centroids, axis=1)

        # 1. Compute cosine similarity
        sim = np.dot(X_norm, centroids_norm.T)
        #print(sim.shape)
        #print(sim)
        # 2. Calculate cosine distance
        cosine_dist = 1 - sim
        #print(cosine_dist.shape)
        #print(cosine_dist)
        # 3. Find maximum cosine distance for each data point
        max_cosine_dist = cosine_dist.max(axis=1)
        #print(max_cosine_dist.shape)
        #print(max_cosine_dist)
        

        # 4. Sum of all maximum distances
        cost = np.sum(max_cosine_dist)

        #cost = np.sum(1 - np.dot(X_norm, centroids_norm.T).max(axis=1))  # Cosine distance = (1 - cosine similarity)  #this code calcs cosine sim to all images? wwhy max? inspect both code each value and then total value
        print(f"Cost function value: {cost:.4f}")  # Print the cost function value

        # Check if this is the best cost so far
        if cost < lowest_cost:
            lowest_cost = cost
            best_centroids = centroids
            best_idx = idx.copy()
            final_iteration = i  # Update the iteration where best centroids were found

    print(f"Final centroids selected from iteration: {final_iteration}")  # Indicate which iteration was chosen
    return best_centroids, best_idx, idx  # Return the best centroids


from sklearn.preprocessing import normalize

# Combine all images into a single dataset for K-Means input
def load_image_data(dataset):
    all_images = []
    all_labels = []
    
    for i in range(len(dataset)):
        flattened_image, label = dataset[i]  # Unpack the image and label
        all_images.append(flattened_image)
        all_labels.append(label)
    
    # Stack all the flattened images to create a large dataset
    X = np.vstack(all_images)  # Combine all images into a single dataset

    # Normalize the combined dataset along the features axis (axis=1)
    X_normalized = normalize(X, axis=1)
    return X_normalized, np.array(all_labels)

# K-Means initialization function
def kMeans_init_centroids(X, K):
    randidx = np.random.permutation(X.shape[0])
    centroids = X[randidx[:K]]  # Randomly select K centroids from normalized X
    return centroids

# Function to find the closest centroids using Cosine Similarity
def find_closest_centroids(X, centroids):
    # No need to normalize X and centroids here as they are already normalized
    idx = np.zeros(X.shape[0], dtype=int)
    for i in range(X.shape[0]):
        similarities = np.dot(centroids, X[i])  # Dot product gives cosine similarity
        idx[i] = np.argmax(similarities)  # We want the most similar (highest value)
    return idx

# Function to compute new centroids
def compute_centroids(X, idx, K):
    centroids = np.zeros((K, X.shape[1]))
    for k in range(K):
        points = X[idx == k]
        if len(points) > 0:
            centroids[k] = np.mean(points, axis=0)  #why axs =0 ans:https://chatgpt.com/share/671b97a7-ec2c-8010-af33-af106df0a25c
            centroids_norm = normalize(centroids, axis=1)
    return centroids_norm

# Function to run K-Means algorithm with cost tracking (using Cosine Similarity)
def run_kMeans(X, initial_centroids, max_iters=10):
    K = initial_centroids.shape[0]
    centroids = initial_centroids
    idx = np.zeros(X.shape[0])

    best_centroids = centroids
    lowest_cost = float('inf')
    final_iteration = 0
    best_idx = idx  # Track the best index assignment

    for i in range(max_iters):
        print(f"K-Means iteration {i}/{max_iters - 1}")
        
        # Assign each data point to the closest centroid
        idx = find_closest_centroids(X, centroids)
        
        # Compute new centroids
        centroids = compute_centroids(X, idx, K)

        # Calculate cost function
        sim = np.dot(X, centroids.T)
        print(f"sim:{sim}")
        cosine_dist = 1 - sim
        #max_cosine_dist = cosine_dist.max(axis=1)
        #cost = np.sum(max_cosine_dist)

        cost = np.sum(cosine_dist)
        

        print(f"Cost function value: {cost}")

        if cost < lowest_cost:
            lowest_cost = cost
            best_centroids = centroids
            best_idx = idx.copy()
            final_iteration = i

    print(f"Final centroids selected from iteration: {final_iteration}")
    return best_centroids, best_idx, centroids, idx  

In [ ]:
# Main function to run the K-Means algorithm
X, labels = load_image_data(dataset)  # Load data and labels 

# To check and print the dimensions:
print("Shape of stacked images (X):", X.shape)
print("Shape of labels:", labels.shape)       

K = 2                          # Step 2: Set number of clusters
initial_centroids = kMeans_init_centroids(X, K)  # Step 3: Initialize centroids 
max_iters = 50                # Step 4: Number of iterations

best_centroids, best_idx, centroids, idx = run_kMeans(X, initial_centroids, max_iters)  # Step 5: Run K-Means
print("Final centroids:", best_centroids)  # Output the final centroids

In [ ]:
print("Cluster assignments:", idx)
print("True labels:", labels)

In [ ]:
print("Cluster assignments:", best_idx)
print("True labels:", labels)

In [ ]:
# `idx` contains the cluster assignments from KMeans
#  `labels` contains the true labels

K = len(np.unique(idx))  # Number of clusters

# Iterate over each cluster and count the true labels in that cluster
for cluster in range(K):
    # Find the indices of images assigned to the current cluster
    cluster_indices = np.where(idx == cluster)[0]
    
    # Get the true labels for the images in this cluster
    cluster_labels = labels[cluster_indices]
    
    # Use Counter to count occurrences of each label in the cluster
    label_counts = Counter(cluster_labels)
    
    # Print the result
    print(f"Cluster {cluster} has true labels: {label_counts}")


In [ ]:
K = len(np.unique(best_idx))  # Number of clusters

# Iterate over each cluster and count the true labels in that cluster
for cluster in range(K):
    # Find the indices of images assigned to the current cluster
    cluster_indices = np.where(best_idx == cluster)[0]
    
    # Get the true labels for the images in this cluster
    cluster_labels = labels[cluster_indices]
    
    # Use Counter to count occurrences of each label in the cluster
    label_counts = Counter(cluster_labels)
    
    # Print the result
    print(f"Cluster {cluster} has true labels: {label_counts}")

In [25]:
# Function to reorder K-Means labels to match true labels
def reorder_labels(true_labels, predicted_labels):
    reordered_labels = np.zeros_like(predicted_labels)

    for cluster in np.unique(predicted_labels):
        mask = (predicted_labels == cluster)
        # Use mode and handle cases where mode() returns a scalar
        most_common_label = mode(true_labels[mask], axis=None).mode  # Get the mode for the current cluster
        if isinstance(most_common_label, np.ndarray):
            most_common_label = most_common_label[0]  # Safely extract the mode value if it's an array
        
        reordered_labels[mask] = most_common_label

    return reordered_labels


In [ ]:
# Apply this after running K-Means
reordered_idx = reorder_labels(labels, idx)

# Now you can compare `reordered_idx` with `labels` to evaluate accuracy
accuracy = np.sum(reordered_idx == labels) / len(labels) * 100
print(f"Accuracy: {accuracy:.2f}%")

In [ ]:
# Apply this after running K-Means
reordered_idx = reorder_labels(labels, best_idx)

# Now you can compare `reordered_idx` with `labels` to evaluate accuracy
accuracy = np.sum(reordered_idx == labels) / len(labels) * 100
print(f"Accuracy: {accuracy:.2f}%")

In [28]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [29]:
# Function to reduce the dimensionality of data points and centroids to 2D using PCA
def reduce_to_2D_pca(X, centroids):
    # Initialize PCA with 2 components
    pca = PCA(n_components=2)
    
    # Apply PCA to the data points (X) and centroids
    X_2D = pca.fit_transform(X)  # Reducing original data points
    centroids_2D = pca.transform(centroids)  # Reducing centroids

    return X_2D, centroids_2D

# Function to plot 2D visualization of clustered data points and centroids with true labels
def plot_2D_clusters_with_labels(X_2D, centroids_2D, idx, labels, K):
    plt.figure(figsize=(10, 7))
    
    # Define color map for clusters
    colors = plt.cm.rainbow(np.linspace(0, 1, K))
    
    # Plot data points colored by their cluster assignment
    for k in range(K):
        cluster_points = X_2D[idx == k]
        cluster_labels = labels[idx == k]  # Get true labels for the current cluster
        
        # Scatter plot for each cluster
        plt.scatter(cluster_points[:, 0], cluster_points[:, 1], 
                    c=[colors[k]], label=f"Cluster {k+1}", alpha=0.6)
        
        # Annotate each point with its true label
        for i in range(cluster_points.shape[0]):
            plt.annotate(str(cluster_labels[i]), 
                         (cluster_points[i, 0], cluster_points[i, 1]), 
                         fontsize=8, alpha=0.75)

    # Plot centroids as larger markers
    plt.scatter(centroids_2D[:, 0], centroids_2D[:, 1], 
                c='k', marker='x', s=200, label='Centroids')

    plt.title("2D Visualization of Clusters using PCA with True Labels")
    plt.xlabel("Component 1")
    plt.ylabel("Component 2")
    plt.legend()
    plt.grid()
    plt.show()


In [ ]:
# Apply PCA after K-Means clustering for 2D visualization
X_2D_pca, centroids_2D_pca = reduce_to_2D_pca(X, centroids)

# Plot the 2D clusters with centroids and true labels
plot_2D_clusters_with_labels(X_2D_pca, centroids_2D_pca, idx, labels, K)


In [ ]:
# Apply PCA after K-Means clustering for 2D visualization
X_2D_pca, centroids_2D_pca = reduce_to_2D_pca(X, best_centroids)

# Plot the 2D clusters with centroids and true labels
plot_2D_clusters_with_labels(X_2D_pca, centroids_2D_pca, best_idx, labels, K)


In [ ]:
total_data_points = 30+30+12
wrong = 3 
predicted_corrected = total_data_points - wrong
accuracy = (predicted_corrected/total_data_points)*100
print(accuracy)

In [33]:
bibi = np.array([
    [0.6394, 0.64353806, 0.64770716, 0.68035775, 0.68076485, 0.68122697],
    [0.6434593, 0.64427465, 0.6459859, 0.6629785, 0.6606025, 0.65433437],
    [0.6341258, 0.63328487, 0.62688166, 0.6681404, 0.66825783, 0.6676554],
    [0.627638, 0.6284246, 0.6292516, 0.61081785, 0.6114242, 0.6106233],
    [0.63396144, 0.63511825, 0.6288564, 0.6226765, 0.6219877, 0.62269145],
    [0.6106442, 0.61703616, 0.6183802, 0.616232, 0.6164079, 0.6152731],
    [0.6106442, 0.61703616, 0.6183802, 0.616232, 0.6164079, 0.6152731]
])

In [ ]:
bibi.shape[0]

In [35]:
class ImageDataset(Dataset):
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.image_files = [os.path.join(image_dir, file) for file in os.listdir(image_dir) if file.endswith('.tiff')] 
        
        # Define a transformation to resize the images
        self.transform = transforms.Compose([transforms.Resize((256, 256))])
     
    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = tiff.imread(img_path)

        # Ensure the image has 3 layers (channels)
        if image.shape[0] != 3:
            raise ValueError(f"Image {img_path} does not have exactly 3 layers.")
        
        # Normalize the 16-bit image to [0, 1]
        original_image = image.astype(np.float32) / 65535.0

        # Convert to a torch tensor 
        image = torch.tensor(original_image, dtype=torch.float32)

        # Apply the resizing transform
        image = self.transform(image)

        image_numpy = image.numpy()

        return image_numpy, original_image


# Create dataset and visualize images
dataset = ImageDataset(r"../../tiff_experiment_unsupervised_data/combined")
for i in range(len(dataset)):
    image_numpy, original_image = dataset[i]  # This will get the resized and original images

    # Plot the original and resized images side by side
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    # Display original image
    axes[0].imshow(original_image.transpose(1, 2, 0))  # Change shape for imshow (H, W, C)
    axes[0].set_title('Original Image')
    axes[0].axis('off')  # Turn off axis

    # Display resized image
    axes[1].imshow(image_numpy.transpose(1, 2, 0))  # Change shape for imshow (H, W, C)
    axes[1].set_title('Resized Image')
    axes[1].axis('off')  # Turn off axis

    plt.tight_layout()
    plt.show()

    break  

class ImageData(Dataset):
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.image_files = [os.path.join(image_dir, file) for file in os.listdir(image_dir) if file.endswith('.tiff')] 
        
        # Define a transformation to resize the images (Note: Resize expects a PIL image or torchvision tensor)
        self.transform = transforms.Resize((256, 256))
     
    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = tiff.imread(img_path)

        # Print the shape of the loaded image
        print(f"Image {img_path} loaded with shape: {image.shape}")

        # Ensure the image has 3 layers (channels)
        if image.shape[0] != 3:
            raise ValueError(f"Image {img_path} does not have exactly 3 layers.")
        
        # Normalize the 16-bit image to [0, 1]
        original_image = image.astype(np.float32) / 65535.0
        print(original_image.shape)
        # Convert to a torch tensor 
        image = torch.tensor(original_image, dtype=torch.float32)
        print(f"tensor image {image.shape}")
        # Apply resizing using torch.nn.functional.interpolate
        image = F.interpolate(image.unsqueeze(0), size=(256, 256), mode='bilinear', align_corners=False).squeeze(0)

        image_numpy = image.numpy()

        # Flatten the resized image for K-Means input (reshape into (3, 256*256))
        #reshaped_image = image.view(3, -1).T  # Reshape to (256*256, 3)
        #print(f"Reshaped image shape: {reshaped_image.shape}")

        return original_image, image_numpy

# Create dataset and visualize images
dataset = ImageData(r"../../tiff_experiment_unsupervised_data/combined")
for i in range(len(dataset)):
    orig_image, image_np = dataset[i]  # This will get the resized and original images

    # Plot the original and resized images side by side
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    # Display original image
    axes[0].imshow(orig_image.transpose(1, 2, 0))  # Change shape for imshow (H, W, C)
    axes[0].set_title('Original Image')
    axes[0].axis('off')  # Turn off axis

    # Display resized image
    axes[1].imshow(image_np.transpose(1, 2, 0))  # Change shape for imshow (H, W, C)
    axes[1].set_title('Resized Image')
    axes[1].axis('off')  # Turn off axis

    plt.tight_layout()
    plt.show()

    break  

image = tiff.imread(r'C:\Users\k54739\Bibin Babu\thesis\Data_supervised\drug_screened\B02-T01.tiff')

# Print the shape of the loaded image
print(f"Image shape {image.shape}")

# Ensure the image has 3 layers (channels)
if image.shape[0] != 3:
    raise ValueError(f"Image {img_path} does not have exactly 3 layers.")
        
# Normalize the 16-bit image to [0, 1]
original_image = image.astype(np.float32) / 65535.0
print(original_image.shape)
# Convert to a torch tensor 
image = torch.tensor(original_image, dtype=torch.float32)




def resize_with_aspect_ratio(image, target_size):
    # Calculate the target size maintaining aspect ratio
    h, w = image.shape[1], image.shape[2]
    aspect_ratio = w / h
    
    if aspect_ratio > 1:  # Wider than tall
        new_width = target_size
        new_height = int(target_size / aspect_ratio)
    else:  # Taller than wide
        new_height = target_size
        new_width = int(target_size * aspect_ratio)

    resized_image = F.interpolate(image.unsqueeze(0), size=(new_height, new_width), mode='bilinear', align_corners=False).squeeze(0)
    
    # Pad to the target size if necessary
    padded_image = F.pad(resized_image, (0, target_size - new_width, 0, target_size - new_height), mode='constant', value=0)
    return padded_image
padded_image = resize_with_aspect_ratio(image,256)
print(padded_image.shape)
np_image = padded_image.numpy()

import numpy as np
import matplotlib.pyplot as plt
import torch
import tifffile as tiff  # Import tifffile for reading TIFF images

# Function to resize with aspect ratio while padding
def resize_with_aspect_ratio(image, target_size):
    # Get original dimensions
    h, w = image.shape[1], image.shape[2]  # Assuming image shape is (C, H, W)
    aspect_ratio = w / h
    
    if aspect_ratio > 1:  # Wider than tall
        new_w = target_size
        new_h = int(target_size / aspect_ratio)
    else:  # Taller than wide or square
        new_h = target_size
        new_w = int(target_size * aspect_ratio)

    # Resize using F.interpolate
    resized_image = torch.nn.functional.interpolate(
        image.unsqueeze(0), size=(new_h, new_w), mode='bilinear', align_corners=False
    ).squeeze(0)

    # Create a padded image
    padded_image = torch.zeros((3, target_size, target_size))  # Create an empty image of target size
    padded_image[:, :new_h, :new_w] = resized_image  # Place resized image in the top-left corner

    return padded_image

# Load the image
img_path = r'C:\Users\k54739\Bibin Babu\thesis\Data_supervised\drug_screened\B02-T01.tiff'
image = tiff.imread(img_path)

# Print the shape of the loaded image
print(f"Image shape: {image.shape}")

# Ensure the image has 3 layers (channels)
if image.shape[0] != 3:
    raise ValueError(f"Image {img_path} does not have exactly 3 layers.")
        
# Normalize the 16-bit image to [0, 1]
original_image = image.astype(np.float32) / 65535.0
print(f"Normalized image shape: {original_image.shape}")

# Convert to a torch tensor 
image_tensor = torch.tensor(original_image, dtype=torch.float32)

# Resize with aspect ratio and pad
padded_image = resize_with_aspect_ratio(image_tensor, 256)
print(f"Padded image shape: {padded_image.shape}")

# Convert padded tensor back to numpy array for visualization
np_image = padded_image.numpy()

# Plot the original and padded images side by side
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Display original image
axes[0].imshow(np.transpose(original_image, (1, 2, 0)))  # Transpose for matplotlib
axes[0].set_title('Original Image')
axes[0].axis('off')

# Display padded image
axes[1].imshow(np.transpose(np_image, (1, 2, 0)))  # Transpose for matplotlib
axes[1].set_title('Padded Image')
axes[1].axis('off')

plt.tight_layout()
plt.show()


dataset = ImageDataset(r"../../tiff_experiment_unsupervised_data/combined")
for i in range(len(dataset)):
    falttened_image = dataset[i]  # This will print the shape of each image
    break

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load an example dataset that we will be using
def load_data():
    return np.random.rand(300, 2) * 10  # 300 data points with 2 features

# Function for random initialization of centroids
def kMeans_init_centroids(X, K):
    randidx = np.random.permutation(X.shape[0])
    centroids = X[randidx[:K]]
    return centroids

# Function to find the closest centroids
def find_closest_centroids(X, centroids):
    idx = np.zeros(X.shape[0], dtype=int)
    for i in range(X.shape[0]):
        distances = np.linalg.norm(X[i] - centroids, axis=1)
        idx[i] = np.argmin(distances)
    return idx

# Function to compute new centroids
def compute_centroids(X, idx, K):
    centroids = np.zeros((K, X.shape[1]))  # Use shape[1] for features
    for k in range(K):
        points = X[idx == k]
        centroids[k] = np.mean(points, axis=0) if len(points) > 0 else centroids[k]
    return centroids

# Function to run K-Means algorithm
def run_kMeans(X, initial_centroids, max_iters=10):
    K = initial_centroids.shape[0]
    centroids = initial_centroids
    idx = np.zeros(X.shape[0])
    centroids_history = []

    for i in range(max_iters):
        print(f"K-Means iteration {i}/{max_iters - 1}")
        # Assign each data point to the closest centroid
        idx = find_closest_centroids(X, centroids)
        # Store the current centroids
        centroids_history.append(centroids.copy())
        # Compute new centroids
        centroids = compute_centroids(X, idx, K)

    # Final plot after all iterations
    plot_progress_kMeans(X, centroids_history, idx, K)
    return centroids, idx

# Updated plotting function
def plot_progress_kMeans(X, centroids_history, idx, K):
    plt.figure(figsize=(8, 6))
    
    # Get colors for each cluster
    colors = plt.cm.rainbow(np.linspace(0, 1, K))
    
    # Plot data points with colors based on their final assignments
    plt.scatter(X[:, 0], X[:, 1], c=colors[idx], alpha=0.6)

    # Plot all centroid movements
    for k in range(K):
        for i in range(len(centroids_history) - 1):
            plt.plot([centroids_history[i][k, 0], centroids_history[i + 1][k, 0]], 
                     [centroids_history[i][k, 1], centroids_history[i + 1][k, 1]], 
                     'k--')  # Draw dashed lines

        # Plot the last centroid position
        plt.scatter(centroids_history[-1][k, 0], centroids_history[-1][k, 1], marker='x', s=200, c='k', label='Final Centroids' if k == 0 else "")

    plt.title("K-Means Clustering Progress")
    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.legend()
    plt.grid()
    plt.xlim(X[:, 0].min() - 1, X[:, 0].max() + 1)
    plt.ylim(X[:, 1].min() - 1, X[:, 1].max() + 1)
    plt.show()

# Main function to run the K-Means algorithm
X = load_data()                 # Step 1: Load dataset
K = 3                           # Step 2: Set number of clusters
initial_centroids = kMeans_init_centroids(X, K)  # Step 3: Initialize centroids
max_iters = 10                  # Step 4: Number of iterations
centroids, idx = run_kMeans(X, initial_centroids, max_iters)  # Step 5: Run K-Means
print("Final centroids:", centroids)  # Output the final centroids


In [ ]:
X.shape

In [ ]:
centroids.shape

In [ ]:
idx

In [ ]:
idx.shape